In [1]:
import os

import pandas as pd

os.environ [ "KMP_DUPLICATE_LIB_OK" ]= "TRUE"

%load_ext autoreload
%matplotlib notebook
%autoreload 2

# 0. what for
1. 用来画图
# 1. import

In [2]:
from common import *
from utils import *
from mydataset import *
from tqdm import tqdm
from my_collate_fn import my_collate_fn_3

import GT_model.GT_2.SA_for_PT_funcs_delta_eq1 as GT_func
from GT_model.GT_2.SA_for_PT_funcs_delta_eq1 import *
from GT_model.GT_3.SA_for_PT_funcs_delta_eq1 import *

import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.io as pio
scaler = GradScaler(backoff_factor = 0.1)

from Config import config_MDN_plot
# from Config import config
import loss
import plot

In [3]:
reload(config_MDN_plot)  # 必须reload！！
reload(loss)    # 必须reload！！
reload(plot)
reload(GT_func)
from loss import cal_metric
from loss import validate
from plot import *
from my_collate_fn import my_collate_fn_3
from utils import *
from models import Conv_1_4

from Config.config_MDN import DefaultConfig

total_train_step = 0
total_test_step = 0

# Range【一般用不到】
a = 0
b = 255

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# 2. 读入模型数据
## 2.1 Load the model and Load the setting

In [5]:
seed = 407
setup_seed(seed)

In [6]:
SPECIFIC = True         # 是否为特定的idx file 画图
SAVE_DATA = False        # save data or not
NORMALIZATION = False
RECAl_METRIC = True     # 是否重新计算metric of GT2 and GT3
SHOW_GT3 = True

In [7]:
if SPECIFIC:
    rangeA = 0
    rangeB = 1
    file_idx = 701  # seed=3: 364, seed=31: 685, seed=204: 460,
else:
    rangeA = 120
    rangeB = 140   # len(test_idx)    # len(test_idx)

NBins = 300   # bins number in the x-axis

In [53]:
MODEL_NAME = "GT1_GT2_EMD"
INPUT_LIST = get_Input_List(MODEL_NAME)
opt = DefaultConfig(MODEL_NAME=MODEL_NAME)
mlp = Conv_1_4(opt.N_gaussians, ch_in=len(INPUT_LIST))
model_path = get_MDN_save_path(opt.ARTIFICIAL, seed, opt.net_root_path, opt.noise_pct, MODEL_NAME)
mlp, _ = load_checkpoint(model_path, mlp)
mlp = mlp.to(device=device)
mlp.eval()

Conv_1_4(
  (BN1): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_pi): Conv_block_4(
    (ac_func): Softplus(beta=1, threshold=20)
    (conv): Conv2d(1, 1, kernel_size=(3, 9), stride=(1, 3))
    (BN_aff2): BatchNorm1d(98, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer_scale): Conv_block_4(
    (ac_func): Softplus(beta=1, threshold=20)
    (conv): Conv2d(1, 1, kernel_size=(3, 9), stride=(1, 3))
    (BN_aff2): BatchNorm1d(98, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer_shape): Conv_block_4(
    (ac_func): Softplus(beta=1, threshold=20)
    (conv): Conv2d(1, 1, kernel_size=(3, 9), stride=(1, 3))
    (BN_aff2): BatchNorm1d(98, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ac_func): Softplus(beta=1, threshold=20)
  (z_pi): Sequential(
    (0): Linear(in_features=98, out_features=2, bias=True)
    (1): Softmax(dim=1)
  )
  (z_scale): Linear(in_features=98, out_feat

In [44]:
dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path, opt.NLL_metric_path)
shuffled_indices = save_data_idx(dataset, opt.arr_flag)
train_idx, val_idx, test_idx = get_data_idx(shuffled_indices,opt.train_pct, opt.vali_pct,opt.SET_VAL)
# Data keys
data_key = pd.read_csv(opt.data_key_path,encoding="utf-8")
# GT-2 params
params_2 = pd.read_csv(opt.params_path_2,encoding="utf-8")
# GT-3 params
params_3 = pd.read_csv(opt.params_path_3,encoding="utf-8")

- 注意repeat之后: [N_samples,Dim]

In [54]:
NLL_test_sum = 0.
GT_metric = torch.tensor([0.,0.,0.,0.]).reshape(1,-1)

for i in tqdm(range(rangeA,rangeB)):
    if SPECIFIC:
        idx_file = file_idx
    train_data, target_data ,_, settings_data, metric_df = dataset.__getitem__(idx_file)

    d, b, v, flg_fixedprice = settings_data   # bidincrement,bidfee,retail,flg_fixedprice

    # 给GT-1和GT-2画图，通过target_data得到U和P，从而画图
    duration = target_data[:,0]
    duration_max = max(duration)

    LEN, T = get_LEN_T(v, b, d, duration_max)

    U_1 = get_U_GT1(LEN, v, d, b,eps = 0.)
    alpha_2 = params_2.iloc[idx_file,0]
    labda_2 = params_2.iloc[idx_file,1]
    U_2 = get_U_GT2(LEN, v, d, b, alpha_2, labda_2,eps = 0.)
    alpha_3 = params_3.iloc[idx_file,0]
    U_3 = get_U_GT3(LEN, v, d, b, alpha_3, eps = 0.)


    # Get x-axis data for GT models
    max_num = min(duration_max,T)
    duration_int = [int(x) for x in duration]
    duration_idx = [int(x) for x in duration if x < max_num]
    duration_idx_index = [i-1 for i in duration_idx]

    P_1 = get_P(U_1,LEN)
    P_1_plot = P_1[duration_idx_index]
    P_2 = get_P(U_2,LEN)
    P_2_plot = P_2[duration_idx_index]
    P_3 = get_P(U_3,LEN)
    P_3_plot = P_3[duration_idx_index]

    # Cal metric
    if RECAl_METRIC:
        GT_2_nll = get_nll_meric(duration, U_2, LEN,TARGET = 1)
        GT_3_nll = get_nll_meric(duration, U_3, LEN,TARGET = 1)
        print("GT-2 metric: ",GT_2_nll)
        print("GT-3 metric: ",GT_3_nll)
    if SHOW_GT3:
        print(f"GT-3 has alpha: {alpha_3}")

    # Get distrb.
    my_collate_fn = functools.partial(my_collate_fn_3, INPUT_LIST=INPUT_LIST)
    _,_,test_loader = get_data_loader(dataset, opt.batch_size, test_idx, test_idx, test_idx, my_collate_fn)
    mlp.eval()
    with torch.no_grad():
        for batch_id, data in enumerate(test_loader):
            input_data, _, _, _, _ = data
            input_data = input_data.to(device)
            pi_idx, scale_idx, shape_idx = mlp(input_data)

    # Cal nll of MDN
    nll = cal_metric(pi_idx.unsqueeze(0), scale_idx.unsqueeze(0), shape_idx.unsqueeze(0), torch.tensor(target_data).unsqueeze(0), opt.N_gaussians, settings_data, opt.MIN_LOSS, device)

    NLL_test_sum += nll.detach().cpu().numpy()
    GT_metric += metric_df

    target_nonzero = np.expand_dims(duration,axis=1)        # For plotting here
    print(f"target_nonzero.shape = {target_nonzero.shape}")

    # 注意repeat的方式不同
    target_nonzero_2 = torch.tensor(target_nonzero.repeat(opt.N_gaussians,1)).to(device=device)
    print(f"target_nonzero_2.shape = {target_nonzero_2.shape}")
    m = torch.distributions.Weibull(scale_idx,shape_idx)
    loss_1 = (m.cdf(target_nonzero_2 + 0.5) - m.cdf(target_nonzero_2 - 0.5)).to(device=device)
    prob_pred = torch.sum(loss_1 * pi_idx, dim=1)
    prob_pred_np = prob_pred.detach().cpu().numpy()

    ### NORMALIZATION
    if NORMALIZATION:
        P_1_plot = P_1_plot/P_1_plot.sum()
        P_2_plot = P_2_plot/P_2_plot.sum()
        P_3_plot = P_3_plot/P_3_plot.sum()
        prob_pred_np = prob_pred_np/prob_pred_np.sum()

    hist_trace = go.Histogram(x=duration, nbinsx=NBins, histnorm='probability',name='target')
    line_trace = go.Scatter(x=duration, y=prob_pred_np, name='pred')
    line_trace_1 = go.Scatter(x=duration_idx, y=P_1_plot, name='GT1')
    line_trace_2 = go.Scatter(x=duration_idx, y=P_2_plot, name='GT2')
    line_trace_3 = go.Scatter(x=duration_idx, y=P_3_plot, name='GT3')

    # Create a layout
    layout = go.Layout(title=f'idx = {idx_file},seed={opt.seed}| GT = {metric_df} | NN = {nll}', xaxis=dict(title='duration'), yaxis=dict(title='density'))
    # Combine the traces into a single figure
    fig = go.Figure(data=[hist_trace, line_trace,line_trace_1,line_trace_2,line_trace_3], layout=layout)

    # Show the figure
    plot_html = fig.to_html(full_html=False)
    fig.show()

    if SAVE_DATA and not opt.ARTIFICIAL:
        path_root_i = "Real_"+"keyid=" + str(file_idx)+"_seed=" + str(opt.seed)
        folder_path = os.path.join(opt.fig_path,path_root_i)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Get full prediction: prob_pred_np
        target_nonzero_data = torch.arange(1,max(duration_int)+1).reshape(-1,1)     # For output data

        target_nonzero_2 = torch.tensor(target_nonzero_data.repeat(opt.N_gaussians,1)).to(device=device)

        loss_1 = (m.cdf(target_nonzero_2 + 0.5) - m.cdf(target_nonzero_2 - 0.5)).to(device=device)
        prob_pred = torch.sum(loss_1 * pi_idx, dim=1)
        prob_pred_np = prob_pred.detach().cpu().numpy()

        assert len(prob_pred_np)==max(duration_int),"Wrong Length"
        # Save the GT data
        GT_data_arr= np.array([0.]*max(duration_int)*6).reshape(-1,6)
        GT_data_i = pd.DataFrame(GT_data_arr,columns=['N','GT1','GT2','GT3','pred','target'])

        # 所有的duration, value从1开始
        # 表格长度取决于duration的长度
        GT_data_i.loc[:,'N'] = np.arange(1,max(duration_int)+1)
        target_p = target_data[:,1]
        # 如果有预测值/实际值，赋值为prob. 否则依然是0
        # 注意这里取的全部的P sequence，而不仅仅是target data中出现的duration的p
        assert len(P_1) == len(P_2)
        for i in range(0,max(duration_idx)):
            GT_data_i.loc[i,'GT1'] = P_1[i]
            GT_data_i.loc[i,'GT2'] = P_2[i]
            GT_data_i.loc[i,'GT3'] = P_3[i]
        # MDN和target data永远可以把表格填满
        for i in range(0,len(GT_data_i)):
            # 注意这里取的全部的P sequence，而不仅仅是target data中出现的duration的p
            GT_data_i.loc[i,'pred'] = prob_pred_np[i]
            # 先获取某个index是否在target上有对应的数据
            if i+1 in target_data[:,0]:
                idx_i = np.where(target_data[:,0]==i+1)[0][0]
                # print(idx_i)
                # 根据idx_i获取对应的p，因为真实数据的缘故，所以这一列其实会有很多0
                GT_data_i.loc[i,'target'] = target_data[idx_i,1]

        file_name = "data.csv"
        file_path = os.path.join(folder_path,file_name)
        GT_data_i.to_csv(file_path,header = True,index = False, encoding="utf-8")

print("Done")

  0%|          | 0/1 [00:00<?, ?it/s]

GT-2 metric:  4.663951403765056
GT-3 metric:  3.7892617773145747
GT-3 has alpha: -0.0062602986581623
target_nonzero.shape = (68, 1)
target_nonzero_2.shape = torch.Size([68, 2])


100%|██████████| 1/1 [00:02<00:00,  2.17s/it]

Done


In [46]:
# GT_metric.shape
NLL_test_sum/len(test_idx),GT_metric/len(test_idx)

(0.02401447095321102,
 tensor([[0.0148, 0.0133, 0.0132, 0.0183]], dtype=torch.float64))

# 4. 重新可视化
1. 读取csv，重新整理画图的间距

In [39]:
idx = 4   # Output file index [1,2,3]
data_path = "D://Desktop/MDN_papaer_results/plot_curve_"+str(idx)+"_v2.csv"
out_path = "D://Desktop/MDN_papaer_results/sum_plot_curve_"+str(idx)+"_v2.csv"

In [40]:
data_plot = pd.read_csv(data_path, encoding="utf-8")
col = data_plot.columns.tolist()
col

['GT1(MDN) (NLL: 7.335)',
 'GT2(MDN) (NLL: 8.416)',
 'EMD (NLL: 7.427)',
 'GT1+EMD (NLL: 7.192)',
 'GT2+EMD (NLL: 7.114)',
 'GT1+GT2 (NLL: 7.352)',
 'GT1_GT2_EMD (NLL: 7.133)']

In [41]:
data_plot.head()

,GT1(MDN) (NLL: 7.335),GT2(MDN) (NLL: 8.416),EMD (NLL: 7.427),GT1+EMD (NLL: 7.192),GT2+EMD (NLL: 7.114),GT1+GT2 (NLL: 7.352),GT1_GT2_EMD (NLL: 7.133)
0,0.003975,0.069138,0.002500,0.000236,0.000553,0.005094,0.000226
1,0.003492,0.037844,0.002748,0.000275,0.000593,0.004341,0.000282
2,0.003243,0.026962,0.002892,0.000301,0.000619,0.003968,0.000320
3,0.003077,0.021187,0.002992,0.000321,0.000640,0.003723,0.000350
4,0.002953,0.017551,0.003067,0.000338,0.000657,0.003543,0.000375


In [48]:
interval = 10 # Sum interval
len_plot = int(np.ceil(data_plot.shape[0]/interval))  # summed_plot的行数
summed_plot_arr = np.array([0]*len(col)*len_plot).reshape(-1,len(col))
summed_plot = pd.DataFrame(summed_plot_arr,columns=col)
summed_plot.shape

(125, 7)

In [49]:
summed_plot.head()

,GT1(MDN) (NLL: 7.335),GT2(MDN) (NLL: 8.416),EMD (NLL: 7.427),GT1+EMD (NLL: 7.192),GT2+EMD (NLL: 7.114),GT1+GT2 (NLL: 7.352),GT1_GT2_EMD (NLL: 7.133)
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0


In [50]:
# 计算每interval行的值之和
idx = 0

# For each line of summed_plot
for i in range(len(summed_plot)):
    start = i*interval
    end = i*interval+interval if i*interval+interval < data_plot.shape[0] else data_plot.shape[0]

    summed_plot.iloc[i,:] = data_plot.iloc[start:end].sum()
    summed_plot.iloc[i,0] = int(start+1)    # 给N单独赋值
print("Done")

Done


In [51]:
summed_plot.head()

,GT1(MDN) (NLL: 7.335),GT2(MDN) (NLL: 8.416),EMD (NLL: 7.427),GT1+EMD (NLL: 7.192),GT2+EMD (NLL: 7.114),GT1+GT2 (NLL: 7.352),GT1_GT2_EMD (NLL: 7.133)
0,0.030299,0.232895,0.030205,0.003358,0.006544,0.036665,0.003711
1,0.023786,0.067514,0.033213,0.004446,0.007619,0.027362,0.005282
2,0.021303,0.042056,0.032980,0.005065,0.008201,0.024013,0.006145
3,0.019693,0.030641,0.031853,0.005528,0.008620,0.021892,0.006771
4,0.018487,0.024076,0.030345,0.005905,0.008946,0.020333,0.007263


In [52]:
summed_plot.to_csv(out_path,encoding="utf-8", header=True, index=False)
out_path,summed_plot.shape

('D://Desktop/MDN_papaer_results/sum_plot_curve_4_v2.csv', (125, 7))

# 5. New Added: 得到其他模型的output用于excel的画图

In [18]:
seed = 407
setup_seed(seed)
# idx_file = 364
test_idx = [701]  # idx needed to be plot

In [19]:
MODEL_LIST = ["GT1(MDN)", "GT2(MDN)", "EMD", "GT1+EMD", "GT2+EMD", "GT1+GT2", "GT1_GT2_EMD"]

# Save the GT data
GT_data_i = pd.DataFrame()

In [23]:
for MODEL_NAME in MODEL_LIST:
    print(f"MODEL_NAME = {MODEL_NAME}")

    INPUT_LIST = get_Input_List(MODEL_NAME)

    opt = DefaultConfig(MODEL_NAME=MODEL_NAME)
    dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path, opt.NLL_metric_path)

    train_data, target_data ,_, settings_data, metric_df = dataset.__getitem__(test_idx[0])
    my_collate_fn = functools.partial(my_collate_fn_3, INPUT_LIST=INPUT_LIST)
    _,_,test_loader = get_data_loader(dataset, opt.batch_size, test_idx, test_idx, test_idx, my_collate_fn)

    # 给GT-1和GT-2画图，通过target_data得到U和P，从而画图
    duration = target_data[:,0]
    duration = [int(x) for x in duration]
    duration_max = max(duration)

    print(f"duration_max = {duration_max}")

    mlp = Conv_1_4(opt.N_gaussians, ch_in=len(INPUT_LIST))
    model_path = get_MDN_save_path(opt.ARTIFICIAL, seed, opt.net_root_path, opt.noise_pct, MODEL_NAME)
    mlp, _ = load_checkpoint(model_path, mlp)
    mlp = mlp.to(device=device)

    mlp.eval()
    with torch.no_grad():
        for batch_id, data in enumerate(test_loader):
            input_data, _, _, _, _ = data
            input_data = input_data.to(device)
            pi_idx, scale_idx, shape_idx = mlp(input_data)

    # Cal NLL
    nll = cal_metric(pi_idx.unsqueeze(0), scale_idx.unsqueeze(0), shape_idx.unsqueeze(0), torch.tensor(target_data).unsqueeze(0), opt.N_gaussians, settings_data, opt.MIN_LOSS, device)

    # To output datapoint
    # Get full prediction: prob_pred_np
    input_nonzero_data = torch.arange(1,duration_max+1).reshape(-1,1)     # For output data
    print(f"input_nonzero_data.shape = {input_nonzero_data.shape}")

    input_nonzero_2 = torch.tensor(input_nonzero_data.repeat(1,opt.N_gaussians)).to(device=device)  # ?
    print(f"input_nonzero_2.shape = {input_nonzero_2.shape}")

    m = torch.distributions.Weibull(scale_idx,shape_idx)
    data_output = (m.cdf(input_nonzero_2 + 0.5) - m.cdf(input_nonzero_2 - 0.5)).to(device=device)
    prob_pred = torch.sum(data_output * pi_idx, dim=1)
    prob_pred_np = prob_pred.detach().cpu().numpy()

    assert len(prob_pred_np)==duration_max,"Wrong Length"

    GT_data_i[MODEL_NAME+ " (NLL: " +str(np.around(nll.detach().cpu().numpy(),decimals=3)) +")"]= prob_pred_np


MODEL_NAME = GT1(MDN)
duration_max = 1247
input_nonzero_data.shape = torch.Size([1247, 1])
input_nonzero_2.shape = torch.Size([1247, 2])
MODEL_NAME = GT2(MDN)
duration_max = 1247
input_nonzero_data.shape = torch.Size([1247, 1])
input_nonzero_2.shape = torch.Size([1247, 2])
MODEL_NAME = EMD
duration_max = 1247
input_nonzero_data.shape = torch.Size([1247, 1])
input_nonzero_2.shape = torch.Size([1247, 2])
MODEL_NAME = GT1+EMD
duration_max = 1247
input_nonzero_data.shape = torch.Size([1247, 1])
input_nonzero_2.shape = torch.Size([1247, 2])
MODEL_NAME = GT2+EMD
duration_max = 1247
input_nonzero_data.shape = torch.Size([1247, 1])
input_nonzero_2.shape = torch.Size([1247, 2])
MODEL_NAME = GT1+GT2
duration_max = 1247
input_nonzero_data.shape = torch.Size([1247, 1])
input_nonzero_2.shape = torch.Size([1247, 2])
MODEL_NAME = GT1_GT2_EMD
duration_max = 1247


C:\Users\Administrator\AppData\Local\Temp\ipykernel_14972\4271937596.py:40: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14972\4271937596.py:40: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14972\4271937596.py:40: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14972\4271937596.py:40: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() 

input_nonzero_data.shape = torch.Size([1247, 1])
input_nonzero_2.shape = torch.Size([1247, 2])


C:\Users\Administrator\AppData\Local\Temp\ipykernel_14972\4271937596.py:40: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [24]:
GT_data_i.head()

,GT1(MDN) (NLL: 7.335),GT2(MDN) (NLL: 8.416),EMD (NLL: 7.427),GT1+EMD (NLL: 7.192),GT2+EMD (NLL: 7.114),GT1+GT2 (NLL: 7.352),GT1_GT2_EMD (NLL: 7.133)
0,0.003975,0.069138,0.002500,0.000236,0.000553,0.005094,0.000226
1,0.003492,0.037844,0.002748,0.000275,0.000593,0.004341,0.000282
2,0.003243,0.026962,0.002892,0.000301,0.000619,0.003968,0.000320
3,0.003077,0.021187,0.002992,0.000321,0.000640,0.003723,0.000350
4,0.002953,0.017551,0.003067,0.000338,0.000657,0.003543,0.000375


In [25]:
path_root_i = "Real_"+"keyid=" + str(file_idx)+"_seed=" + str(seed)
folder_path = os.path.join(opt.fig_path,path_root_i)

In [26]:
file_name = "data_other_models.csv"
file_path = os.path.join(folder_path,file_name)
GT_data_i.to_csv(file_path,header = True,index = False, encoding="utf-8")